# 用Word2Vec做自然语言分析

## 准备语料

网络上高质量的语料比较少，中文的高质量语料就更少了，我在知乎的专栏上找到一篇文章总结了[各个领域的公开数据集](https://zhuanlan.zhihu.com/p/25138563)。

在本文中我们使用的是[搜狗新闻语料数据](https://www.sogou.com/labs/resource/list_news.php)，

我下载的新闻语料压缩包大小在640MB，解压后是一个dat格式的文件，数据格式为：

```xml
<doc>
<url>页面URL</url>
<docno>页面ID</docno>
<contenttitle>页面标题</contenttitle>
<content>页面内容</content>
</doc>
```

原始的语料数据是GBK格式的，在linux上是乱码，需要将其转换为UTF-8编码，顺带的我们也只需要`<content>`标签内的内容，因此可以一并将其处理了。使用shell命令可以一步完成：

```shell
cat news_tensite_xml.dat | iconv -f gbk -t utf-8 -c | grep "<content>"  > sougou_corpus.txt 
```

执行命令后效果如图：

![sourgou_corpus.txt](img/sougou_corpus.png)



## 分词

准备好语料之后，接下来需要做的是分词了。这一步骤对于中文很重要，因为英文是由一个个单词组成，而中文是由字组成，很多时间我们需要由字组成词之后才有意义。

中文的NLP处理中有一些用来做分词的工具，在这里我们使用[结巴分词](https://github.com/fxsjy/jieba)。

In [1]:
#-*-coding:utf-8-*- 
import logging
import os.path
import jieba
import sys
import re

In [3]:
# 用正则表达式提取content
def removeContext(content):
    reContent = re.sub('<content>|</content>', '', content)
    return reContent

In [2]:
# 初始化日志配置
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)
# logging.info("running %s" % ' '.join(sys.argv))

In [3]:
# 获取当前文件所在绝对路径
base_path = os.path.abspath('.')
dataset_path = base_path + '/' + 'dataset'
model_path = base_path + '/' + 'model'

input_corpus = dataset_path + '/' + 'sougou_corpus.txt'
seg_corpus = dataset_path + '/' + 'seg_sougou_corpus.txt'

word2vec_model = model_path + '/' + 'word2vec_gen'

In [16]:
punct = set(u''':!),.:;?]}¢'"、。〉》」』】〕〗〞︰︱︳﹐､﹒
﹔﹕﹖﹗﹚﹜﹞！），．：；？｜｝︴︶︸︺︼︾﹀﹂﹄﹏､～￠
々‖•·ˇˉ―--′’”([{£¥'"‵〈《「『【〔〖（［｛￡￥〝︵︷︹︻
︽︿﹁﹃﹙﹛﹝（｛“‘-—_…''')

# 去除特殊字符
filterpunt = lambda l: list(filter(lambda x: x not in punct, l))

i = 0
try:
    fo = open(seg_corpus, 'w', encoding='utf-8')
    with open(input_corpus, 'r', encoding='utf-8') as f:
        for line in f:
            line_seg = list(jieba.cut(removeContent(line), cut_all=False))
            line_seg = filterpunt(line_seg)
            [word for word in line_seg if word != ''] #去除空格
            fo.write(' '.join(line_seg))  # 分好的词用空格连接
            fo.write('\n\r')
            i = i + 1
            if (i % 10000 == 0):
                logger.info("Saved " + str(i))
finally:
    fo.close()
        
logger.info("Finished Saved " + str(i))
# finput = open(dataset_path)

Building prefix dict from the default dictionary ...
2018-04-17 22:17:30,288: DEBUG: Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
2018-04-17 22:17:30,291: DEBUG: Loading model from cache /tmp/jieba.cache
Loading model cost 1.615 seconds.
2018-04-17 22:17:31,905: DEBUG: Loading model cost 1.615 seconds.
Prefix dict has been built succesfully.
2018-04-17 22:17:31,907: DEBUG: Prefix dict has been built succesfully.
2018-04-17 22:18:08,713: INFO: Saved 10000
2018-04-17 22:18:48,057: INFO: Saved 20000
2018-04-17 22:19:24,854: INFO: Saved 30000
2018-04-17 22:20:03,984: INFO: Saved 40000
2018-04-17 22:20:41,316: INFO: Saved 50000
2018-04-17 22:21:18,775: INFO: Saved 60000
2018-04-17 22:21:55,472: INFO: Saved 70000
2018-04-17 22:22:30,392: INFO: Saved 80000
2018-04-17 22:23:05,705: INFO: Saved 90000
2018-04-17 22:23:45,710: INFO: Saved 100000
2018-04-17 22:24:25,828: INFO: Saved 110000
2018-04-17 22:25:07,726: INFO: Saved 120000
2018-04-17 22:2

分词后的效果图：

![seg_sourgou_corpus.txt](img/seg_sougou_corpus.png)

## 训练词向量

当我们准备分完词之后就可以开始训练词向量了，在这里我使用的是`gensim`这个`python`的工具包，使用起来非常简单。

In [4]:
from gensim.models import word2vec
from gensim.models import KeyedVectors as kvec
import multiprocessing
import time

2018-04-20 08:52:35,904: INFO: 'pattern' package not found; tag filters are not available for English


In [5]:
base_path = os.path.abspath('.')
dataset_path = base_path + '/' + 'dataset'
output_corpus = dataset_path + '/' + 'seg_sougou_corpus.txt'

words_list = []

foutput = open(output_corpus, 'r', encoding='utf-8')
logger.info(foutput.readline())
foutput.close()

在`gensim`中有一些关于算法的参数可以进行调整

| 参数 | 说明 |
| :--- | :--- |
| sentences | 我们要分析的语料，可以是一个列表，或者是从文件中遍历读出，需要注意的是我们需要输入已经分词的数据 |
| size | 词向量的维度，或者说是神经网络的层数，默认值为100。更大的维度意味着更多的输入数据，不过也能提升整体的准确度。这个维度的取值与我们语料大小有关，如果语料不大，如小于100M，则使用默认值就可以了，如果是超大语料，最好增加维度。 |
| window | 词向量上下文最大距离。通俗的讲是前后看词的单位，3表示在目标词前看3-b个词，后面看b个词(b在0-3之间随机)，如果设置过大也会让结果并不那么理想。|
| sg | 0为CBOW模型计算近似词，1为Skip-Gram模型，对低频词敏感。默认为0 |
| negative | 使用Negative Sampling时负采样个数，默认是5，推荐在[3,10]之间，根据结果进行微调|
| sample | |
| iter | 随机梯度下降算法中迭代的最大次数，默认为5。对于大语料，可以增大这个值 |
| min-count | 需要计算词向量的最小词频，默认是5.这个值可以去掉一些很生僻的词。如果是小语料，可以调低这个值 |
| workers |  |



In [7]:
begin = time.time()
# 读取分好词的文本
sentences = word2vec.LineSentence(seg_corpus)
# 训练词向量
model = word2vec.Word2Vec(sentences, hs=1, 
                                                  min_count=10, window=10,
                                                  size=300, iter=8, 
                                                  workers=multiprocessing.cpu_count())
# 保存模型
model.save(word2vec_model)
model.wv.save_word2vec_format(model_path + '/' + 'word2vec_org',
                                  model_path + '/' + 'vocabulary',
                                  binary=False)

end = time.time()
logging.info("Total procesing time: %d seconds" % (end - begin))

2018-04-19 09:22:03,030: INFO: collecting all words and their counts
2018-04-19 09:22:03,049: INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-19 09:22:04,292: INFO: PROGRESS: at sentence #10000, processed 2382969 words, keeping 114263 word types
2018-04-19 09:22:05,489: INFO: PROGRESS: at sentence #20000, processed 4848010 words, keeping 163620 word types
2018-04-19 09:22:06,622: INFO: PROGRESS: at sentence #30000, processed 7282733 words, keeping 198358 word types
2018-04-19 09:22:07,733: INFO: PROGRESS: at sentence #40000, processed 9682867 words, keeping 226219 word types
2018-04-19 09:22:08,867: INFO: PROGRESS: at sentence #50000, processed 12083765 words, keeping 251611 word types
2018-04-19 09:22:10,029: INFO: PROGRESS: at sentence #60000, processed 14397292 words, keeping 271265 word types
2018-04-19 09:22:11,090: INFO: PROGRESS: at sentence #70000, processed 16661053 words, keeping 291866 word types
2018-04-19 09:22:12,283: INFO: PROGRESS: at sen

2018-04-19 09:23:25,378: INFO: PROGRESS: at sentence #720000, processed 174743016 words, keeping 774184 word types
2018-04-19 09:23:26,571: INFO: PROGRESS: at sentence #730000, processed 177364220 words, keeping 777350 word types
2018-04-19 09:23:27,648: INFO: PROGRESS: at sentence #740000, processed 179762724 words, keeping 780341 word types
2018-04-19 09:23:28,674: INFO: PROGRESS: at sentence #750000, processed 182098742 words, keeping 782443 word types
2018-04-19 09:23:29,684: INFO: PROGRESS: at sentence #760000, processed 184419550 words, keeping 784418 word types
2018-04-19 09:23:30,723: INFO: PROGRESS: at sentence #770000, processed 186710350 words, keeping 787248 word types
2018-04-19 09:23:31,828: INFO: PROGRESS: at sentence #780000, processed 189160155 words, keeping 790208 word types
2018-04-19 09:23:32,719: INFO: PROGRESS: at sentence #790000, processed 191286746 words, keeping 791481 word types
2018-04-19 09:23:33,874: INFO: PROGRESS: at sentence #800000, processed 19385317

2018-04-19 09:25:06,800: INFO: EPOCH 1 - PROGRESS: at 0.21% examples, 110771 words/s, in_qsize 7, out_qsize 0
2018-04-19 09:25:07,858: INFO: EPOCH 1 - PROGRESS: at 0.26% examples, 114048 words/s, in_qsize 7, out_qsize 0
2018-04-19 09:25:08,903: INFO: EPOCH 1 - PROGRESS: at 0.30% examples, 116317 words/s, in_qsize 7, out_qsize 0
2018-04-19 09:25:09,978: INFO: EPOCH 1 - PROGRESS: at 0.35% examples, 117719 words/s, in_qsize 7, out_qsize 0
2018-04-19 09:25:11,016: INFO: EPOCH 1 - PROGRESS: at 0.39% examples, 119172 words/s, in_qsize 7, out_qsize 0
2018-04-19 09:25:12,078: INFO: EPOCH 1 - PROGRESS: at 0.44% examples, 120118 words/s, in_qsize 7, out_qsize 0
2018-04-19 09:25:13,115: INFO: EPOCH 1 - PROGRESS: at 0.48% examples, 121288 words/s, in_qsize 7, out_qsize 0
2018-04-19 09:25:14,173: INFO: EPOCH 1 - PROGRESS: at 0.53% examples, 122067 words/s, in_qsize 6, out_qsize 1
2018-04-19 09:25:15,203: INFO: EPOCH 1 - PROGRESS: at 0.58% examples, 122910 words/s, in_qsize 7, out_qsize 0
2018-04-19

2018-04-19 09:26:24,891: INFO: EPOCH 1 - PROGRESS: at 3.97% examples, 131218 words/s, in_qsize 7, out_qsize 0
2018-04-19 09:26:25,899: INFO: EPOCH 1 - PROGRESS: at 4.02% examples, 131296 words/s, in_qsize 7, out_qsize 0
2018-04-19 09:26:26,908: INFO: EPOCH 1 - PROGRESS: at 4.06% examples, 131360 words/s, in_qsize 7, out_qsize 0
2018-04-19 09:26:27,987: INFO: EPOCH 1 - PROGRESS: at 4.11% examples, 131316 words/s, in_qsize 7, out_qsize 0
2018-04-19 09:26:29,040: INFO: EPOCH 1 - PROGRESS: at 4.16% examples, 131309 words/s, in_qsize 6, out_qsize 1
2018-04-19 09:26:30,093: INFO: EPOCH 1 - PROGRESS: at 4.23% examples, 131400 words/s, in_qsize 7, out_qsize 0
2018-04-19 09:26:31,118: INFO: EPOCH 1 - PROGRESS: at 4.28% examples, 131431 words/s, in_qsize 7, out_qsize 0
2018-04-19 09:26:32,137: INFO: EPOCH 1 - PROGRESS: at 4.32% examples, 131469 words/s, in_qsize 7, out_qsize 0
2018-04-19 09:26:33,164: INFO: EPOCH 1 - PROGRESS: at 4.37% examples, 131490 words/s, in_qsize 7, out_qsize 0
2018-04-19

2018-04-19 09:27:43,013: INFO: EPOCH 1 - PROGRESS: at 7.81% examples, 132813 words/s, in_qsize 7, out_qsize 0
2018-04-19 09:27:44,020: INFO: EPOCH 1 - PROGRESS: at 7.86% examples, 132829 words/s, in_qsize 7, out_qsize 0
2018-04-19 09:27:45,020: INFO: EPOCH 1 - PROGRESS: at 7.89% examples, 132808 words/s, in_qsize 7, out_qsize 0
2018-04-19 09:27:46,073: INFO: EPOCH 1 - PROGRESS: at 7.94% examples, 132797 words/s, in_qsize 6, out_qsize 1
2018-04-19 09:27:47,127: INFO: EPOCH 1 - PROGRESS: at 7.99% examples, 132827 words/s, in_qsize 7, out_qsize 0
2018-04-19 09:27:48,208: INFO: EPOCH 1 - PROGRESS: at 8.04% examples, 132801 words/s, in_qsize 6, out_qsize 1
2018-04-19 09:27:49,221: INFO: EPOCH 1 - PROGRESS: at 8.08% examples, 132814 words/s, in_qsize 7, out_qsize 0
2018-04-19 09:27:50,231: INFO: EPOCH 1 - PROGRESS: at 8.13% examples, 132839 words/s, in_qsize 7, out_qsize 0
2018-04-19 09:27:51,321: INFO: EPOCH 1 - PROGRESS: at 8.17% examples, 132844 words/s, in_qsize 6, out_qsize 1
2018-04-19

...

2018-04-19 13:39:07,369: INFO: EPOCH 8 - PROGRESS: at 82.45% examples, 141836 words/s, in_qsize 8, out_qsize 0
2018-04-19 13:39:08,452: INFO: EPOCH 8 - PROGRESS: at 82.50% examples, 141832 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:39:09,476: INFO: EPOCH 8 - PROGRESS: at 82.56% examples, 141833 words/s, in_qsize 8, out_qsize 1
2018-04-19 13:39:10,477: INFO: EPOCH 8 - PROGRESS: at 82.62% examples, 141844 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:39:11,492: INFO: EPOCH 8 - PROGRESS: at 82.68% examples, 141841 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:39:12,541: INFO: EPOCH 8 - PROGRESS: at 82.76% examples, 141853 words/s, in_qsize 7, out_qsize 1
2018-04-19 13:39:13,592: INFO: EPOCH 8 - PROGRESS: at 82.84% examples, 141864 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:39:14,668: INFO: EPOCH 8 - PROGRESS: at 82.89% examples, 141862 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:39:15,711: INFO: EPOCH 8 - PROGRESS: at 82.95% examples, 141860 words/s, in_qsize 8, out_qsize 0
2

2018-04-19 13:40:24,068: INFO: EPOCH 8 - PROGRESS: at 86.48% examples, 141945 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:40:25,071: INFO: EPOCH 8 - PROGRESS: at 86.54% examples, 141947 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:40:26,203: INFO: EPOCH 8 - PROGRESS: at 86.58% examples, 141938 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:40:27,213: INFO: EPOCH 8 - PROGRESS: at 86.63% examples, 141939 words/s, in_qsize 8, out_qsize 0
2018-04-19 13:40:28,230: INFO: EPOCH 8 - PROGRESS: at 86.68% examples, 141940 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:40:29,269: INFO: EPOCH 8 - PROGRESS: at 86.74% examples, 141942 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:40:30,293: INFO: EPOCH 8 - PROGRESS: at 86.79% examples, 141943 words/s, in_qsize 6, out_qsize 2
2018-04-19 13:40:31,325: INFO: EPOCH 8 - PROGRESS: at 86.84% examples, 141940 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:40:32,333: INFO: EPOCH 8 - PROGRESS: at 86.88% examples, 141936 words/s, in_qsize 7, out_qsize 0
2

2018-04-19 13:41:41,258: INFO: EPOCH 8 - PROGRESS: at 90.14% examples, 141897 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:41:42,268: INFO: EPOCH 8 - PROGRESS: at 90.18% examples, 141894 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:41:43,281: INFO: EPOCH 8 - PROGRESS: at 90.22% examples, 141891 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:41:44,303: INFO: EPOCH 8 - PROGRESS: at 90.26% examples, 141887 words/s, in_qsize 8, out_qsize 0
2018-04-19 13:41:45,329: INFO: EPOCH 8 - PROGRESS: at 90.31% examples, 141883 words/s, in_qsize 8, out_qsize 0
2018-04-19 13:41:46,342: INFO: EPOCH 8 - PROGRESS: at 90.35% examples, 141880 words/s, in_qsize 6, out_qsize 1
2018-04-19 13:41:47,357: INFO: EPOCH 8 - PROGRESS: at 90.39% examples, 141877 words/s, in_qsize 7, out_qsize 1
2018-04-19 13:41:48,414: INFO: EPOCH 8 - PROGRESS: at 90.44% examples, 141875 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:41:49,447: INFO: EPOCH 8 - PROGRESS: at 90.49% examples, 141874 words/s, in_qsize 7, out_qsize 1
2

2018-04-19 13:42:58,164: INFO: EPOCH 8 - PROGRESS: at 93.73% examples, 141785 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:42:59,210: INFO: EPOCH 8 - PROGRESS: at 93.81% examples, 141802 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:43:00,231: INFO: EPOCH 8 - PROGRESS: at 93.88% examples, 141812 words/s, in_qsize 8, out_qsize 0
2018-04-19 13:43:01,239: INFO: EPOCH 8 - PROGRESS: at 93.92% examples, 141805 words/s, in_qsize 8, out_qsize 0
2018-04-19 13:43:02,264: INFO: EPOCH 8 - PROGRESS: at 93.97% examples, 141805 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:43:03,274: INFO: EPOCH 8 - PROGRESS: at 94.02% examples, 141802 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:43:04,276: INFO: EPOCH 8 - PROGRESS: at 94.06% examples, 141800 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:43:05,376: INFO: EPOCH 8 - PROGRESS: at 94.11% examples, 141799 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:43:06,510: INFO: EPOCH 8 - PROGRESS: at 94.18% examples, 141802 words/s, in_qsize 7, out_qsize 0
2

2018-04-19 13:44:15,058: INFO: EPOCH 8 - PROGRESS: at 97.53% examples, 141719 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:44:16,084: INFO: EPOCH 8 - PROGRESS: at 97.58% examples, 141715 words/s, in_qsize 8, out_qsize 0
2018-04-19 13:44:17,098: INFO: EPOCH 8 - PROGRESS: at 97.66% examples, 141722 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:44:18,118: INFO: EPOCH 8 - PROGRESS: at 97.70% examples, 141718 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:44:19,157: INFO: EPOCH 8 - PROGRESS: at 97.75% examples, 141717 words/s, in_qsize 7, out_qsize 1
2018-04-19 13:44:20,208: INFO: EPOCH 8 - PROGRESS: at 97.82% examples, 141726 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:44:21,234: INFO: EPOCH 8 - PROGRESS: at 97.87% examples, 141726 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:44:22,318: INFO: EPOCH 8 - PROGRESS: at 97.95% examples, 141735 words/s, in_qsize 7, out_qsize 0
2018-04-19 13:44:23,405: INFO: EPOCH 8 - PROGRESS: at 98.00% examples, 141730 words/s, in_qsize 7, out_qsize 0
2

我使用了一台i3 4核，4G内存的老笔记本做训练，一共用了4个多小时才训练完，这还是在4核都用上的情况下。而word2vec最大的特点就是速度足够快，无法想象如果没有强劲的处理计算机，个人训练模型会花多少时间。

训练完成之后我们可以看下词向量的最常见的应用，找出最相近的词

In [5]:
model = kvec.load(word2vec_model)
model.most_similar(u'地铁')


2018-04-20 08:52:48,821: INFO: loading Word2VecKeyedVectors object from /home/welljoint/workspace/learning-ai-from-now/Part_3_Practice/model/word2vec_gen
2018-04-20 08:52:52,739: INFO: loading wv recursively from /home/welljoint/workspace/learning-ai-from-now/Part_3_Practice/model/word2vec_gen.wv.* with mmap=None
2018-04-20 08:52:52,747: INFO: loading vectors from /home/welljoint/workspace/learning-ai-from-now/Part_3_Practice/model/word2vec_gen.wv.vectors.npy with mmap=None
2018-04-20 08:52:55,962: INFO: setting ignored attribute vectors_norm to None
2018-04-20 08:52:55,966: INFO: loading vocabulary recursively from /home/welljoint/workspace/learning-ai-from-now/Part_3_Practice/model/word2vec_gen.vocabulary.* with mmap=None
2018-04-20 08:52:55,968: INFO: loading trainables recursively from /home/welljoint/workspace/learning-ai-from-now/Part_3_Practice/model/word2vec_gen.trainables.* with mmap=None
2018-04-20 08:52:55,970: INFO: loading syn1 from /home/welljoint/workspace/learning-ai-fr

[('轻轨', 0.6082177758216858),
 ('轨道交通', 0.6061943769454956),
 ('换乘', 0.582024335861206),
 ('公交', 0.5754963159561157),
 ('轨交', 0.5576555132865906),
 ('北京地铁', 0.5498325824737549),
 ('地铁站', 0.5115457773208618),
 ('号线', 0.4854784607887268),
 ('巴士', 0.45425480604171753),
 ('公共交通', 0.4541604518890381)]

In [11]:
from collections import Counter
total_word_counts = Counter()
with open(seg_corpus, 'r', encoding='utf-8') as f:
    for line in f:
        # 统计词频
        total_word_counts.update(line.split())